# Digit Recognizer / Convolutional Neural Network
## This is my first attempt at a CNN project. 


1.  [Importing](#1)
    1. [Importing the Libraries](#2)
    1. [Importing the Data](#3)
1.  [Labeling & Inspecting](#4)
1.  [Normalizing & Reshaping](#5)
1.  [Label Encoding & Train-Test Split](#6)
1.  [Creating the CNN model using Keras](#7)
1.  [Evaluation & Confusion Matrix](#8)

<a id = "1"></a> 
# 1) Importing 

<a id = "2"></a> 
## Importing the Libraries

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

<a id = "3"></a> 
## Importing the Data

In [ ]:
train = pd.read_csv("../input/digit-recognizer/train.csv")
print(train.shape)
train.head()
# here, the images are 28 by 28,
# 28 * 28 = 784

In [ ]:
test= pd.read_csv("../input/digit-recognizer/test.csv")
print(test.shape)
test.head()

<a id = "4"></a> 
## 2) Labeling & Inspecting

In [ ]:
# put labels into y_train variable

Y_train = train["label"]

# Drop 'label' column
X_train = train.drop(labels = ["label"],axis = 1) 

In [ ]:
# visualize number of digits classes
plt.figure(figsize=(15,7))
g = sns.countplot(Y_train, palette="icefire")
plt.title("Number of digit classes")
Y_train.value_counts()

In [ ]:
# plot some samples
img = X_train.iloc[312].values
img = img.reshape((28,28))
plt.imshow(img,cmap='gray')
plt.title(train.iloc[0,0])
plt.axis("off")
plt.show()

<a id = "5"></a> 
## 3) Normalizing & Reshaping

In [ ]:
# Normalize the data
X_train = X_train / 255.0
test = test / 255.0
print("x_train shape: ",X_train.shape)
print("test shape: ",test.shape)

In [ ]:
# Reshape
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)
print("x_train shape: ",X_train.shape)
print("test shape: ",test.shape)

<a id = "6"></a> 
## 4) Label Encoding & Train-Test Split

In [ ]:
from keras.utils.np_utils import to_categorical 

# convert to one-hot-encoding

Y_train = to_categorical(Y_train, num_classes = 10)

In [ ]:
# Split the train and the validation set for the fitting

from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=2)
print("x_train shape",X_train.shape)
print("x_test shape",X_val.shape)
print("y_train shape",Y_train.shape)
print("y_test shape",Y_val.shape)

<a id = "7"></a> 
## 5) Creating the CNN model using Keras

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

model = Sequential()

model.add(Conv2D(filters = 8, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 16, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))

model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))

model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [ ]:
# Defining the optimizer

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

In [ ]:
# Compiling the model

model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
epochs = 15  # for better result increase the epochs
batch_size = 250

In [ ]:
X_train.shape

In [ ]:
# data augmentation
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # dimesion reduction
        rotation_range=5,  # randomly rotate images in the range 5 degrees
        zoom_range = 0.1, # Randomly zoom image 10%
        width_shift_range=0.1,  # randomly shift images horizontally 10%
        height_shift_range=0.1,  # randomly shift images vertically 10%
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(X_train)

In [ ]:
test_my_model = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_val,Y_val), steps_per_epoch=X_train.shape[0] // batch_size)

<a id = "8"></a> 
## 6) Evaluation / Confusion Matrix

In [ ]:
# Plot the loss and accuracy curves for training and validation 
plt.plot(test_my_model.history['val_loss'], color='b', label="validation loss")
plt.title("Test Loss")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
# confusion matrix
import seaborn as sns
# Predict the values from the validation dataset
Y_pred = model.predict(X_val)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(Y_val,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
f,ax = plt.subplots(figsize=(8, 8))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap="Reds",linecolor="gray", fmt= '.1f',ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Read test data for submission of results
test1 = pd.read_csv("../input/digit-recognizer/test.csv")

# Reshape and scale figures
test1 = test1.values.reshape(28000, 28, 28, 1)
test1 = test1 / 255.0

In [ ]:
# Save results to csv file for submission

df_pred = pd.DataFrame(Y_pred)
df_pred.index = range(1, 28001)
df_pred = df_pred.reset_index()
df_pred.head()
df_pred.columns = ["ImageId", "Label"]
df_pred.to_csv("submission.csv", index=False)

## This was my first attempt at a CNN project. 

### Tutorial from [@DATAI](https://www.kaggle.com/kanncaa1) has been very helpful.
